# Aug 28, 2025: visualize membership matrices 
on brain, per solution mode, per animal

In [1]:
import csv
import os
import sys
import numpy as np
import pandas as pd
import scipy as sp 
import dill as pickle 
from os.path import join as pjoin
from itertools import product
from tqdm import tqdm
from copy import deepcopy
from pathlib import Path
import subprocess
from scipy import sparse, stats
from multiprocessing import Pool
import glob
import random

# import arviz as az

import ants
from nipype.interfaces import afni

from itertools import product, combinations, chain
import multiprocessing as mp
from functools import partial

# networks
import graph_tool.all as gt

# plotting
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.cm import rainbow

plt.rcParamsDefault['font.family'] = "sans-serif"
plt.rcParamsDefault['font.sans-serif'] = "Arial"
plt.rcParams['font.size'] = 14
plt.rcParams["errorbar.capsize"] = 0.5

import cmasher as cmr  # CITE ITS PAPER IN YOUR MANUSCRIPT
import colorcet as cc

# ignore user warnings
import warnings
warnings.filterwarnings("ignore") #, category=UserWarning)

In [2]:
class ARGS():
    pass

args = ARGS()

args.SEED = 100

def set_seed(args):
    gt.seed_rng(args.SEED)
    np.random.seed(args.SEED)

set_seed(args)

In [3]:
args.source ='allen' #'spatial' #'allen'
args.space = 'ccfv2' #'yale' #'ccfv2'
args.brain_div = 'whl' #'whl' #'whl'
args.num_rois = 172 #162 #172
args.resolution = 200 #200 #200

PARC_DESC = (
    f'source-{args.source}'
    f'_space-{args.space}'
    f'_braindiv-{args.brain_div}'
    f'_nrois-{args.num_rois}'
    f'_res-{args.resolution}'
)
PARC_DESC

'source-allen_space-ccfv2_braindiv-whl_nrois-172_res-200'

In [4]:
args.high_res = 25

PARC_DESC_HR = ( # High Resolution
    f'source-{args.source}'
    f'_space-{args.space}'
    f'_braindiv-{args.brain_div}'
    f'_nrois-{args.num_rois}'
    f'_res-{args.high_res}'
)
PARC_DESC_HR

'source-allen_space-ccfv2_braindiv-whl_nrois-172_res-25'

In [5]:
args.GRAPH_DEF = f'constructed'
args.GRAPH_METHOD = f'pearson'
args.THRESHOLD = f'signed'
args.EDGE_DEF = f'binary'
args.EDGE_DENSITY = 20
args.LAYER_DEF = f'individual'
args.DATA_UNIT = f'sub'

BASE_path = f'{os.environ["HOME"]}/new_mouse_dataset'
PARCELS_path = f'{BASE_path}/parcels'
ROI_path = (
    f'{BASE_path}/roi-results-v3'
    f'/{PARC_DESC}'
)
TS_path = f'{ROI_path}/roi_timeseries'
ROI_RESULTS_path = (
    f'{ROI_path}'
    f'/graph-{args.GRAPH_DEF}/method-{args.GRAPH_METHOD}'
    f'/threshold-{args.THRESHOLD}/edge-{args.EDGE_DEF}/density-{args.EDGE_DENSITY}'
    f'/layer-{args.LAYER_DEF}/unit-{args.DATA_UNIT}'
)
GRAPH_path = f'{ROI_RESULTS_path}/graphs'
os.system(f'mkdir -p {GRAPH_path}')
SBM_path = f'{ROI_RESULTS_path}/model-fits'
os.system(f'mkdir -p {SBM_path}')
DIAG_path = f'{ROI_RESULTS_path}/diagnostics'
os.system(f'mkdir -p {DIAG_path}')
ESTIM_path = f'{ROI_RESULTS_path}/estimates'
os.system(f'mkdir -p {ESTIM_path}/individual')
os.system(f'mkdir -p {ESTIM_path}/group')

0

In [6]:
parcels_img = ants.image_read(f'{PARCELS_path}/{PARC_DESC}_desc-parcels.nii.gz')

try:
    roi_labels = pd.read_csv(f'{PARCELS_path}/{PARC_DESC}_desc-names.csv')['roi'].to_list()
except:
    roi_labels = np.arange(1, args.num_rois+1)

In [7]:
# parcels_hr_img = ants.image_read(f'{PARCELS_path}/{PARC_DESC_HR}_desc-parcels.nii.gz')

# roi_labels = pd.read_csv(f'{PARCELS_path}/{PARC_DESC_HR}_desc-names.csv')['roi'].to_list()

In [8]:
parcels_img

ANTsImage (RAI)
	 Pixel Type : float (float32)
	 Components : 1
	 Dimensions : (57, 40, 66)
	 Spacing    : (0.2, 0.2, 0.2)
	 Origin     : (5.78, 7.83, -6.93)
	 Direction  : [1. 0. 0. 0. 1. 0. 0. 0. 1.]

In [9]:
def concatenate(in_files, out_file):
    try:
        os.remove(out_file)
    except:
        pass

    tcat = afni.TCat()
    tcat.inputs.in_files = in_files
    tcat.inputs.out_file = out_file
    tcat.inputs.rlt = ''
    tcat.cmdline 
    tcat.run()

    for file in in_files:
        try:
            os.remove(file)
        except:
            pass
    return None

def marginal_to_nifti(args, parcels_img, X, mode_id, folder, level=-1):
    parcels_vol = parcels_img.numpy()
    parcels_vol = np.round(parcels_vol, decimals=0)

    os.system(f'mkdir -p {folder}')
    args.num_rois, args.num_comms = X.shape

    in_files = []
    for idx_comm in range(args.num_comms):
        x = X[:, idx_comm]
        x_img = np.zeros_like(parcels_vol)
        for idx, roi in enumerate(roi_labels):
            x_img += (parcels_vol == roi) * (x[idx])
        
        file = f'{folder}/mode-{mode_id}_comm-{idx_comm}.nii.gz'
        parcels_img.new_image_like(x_img).to_filename(file)
        in_files.append(file)

    if level == -1:
        out_file = f'{folder}/mode-{mode_id:02d}.nii.gz'
    else:
        out_file = f'{folder}/mode-{mode_id:02d}_level-{level}.nii.gz'
    concatenate(in_files, out_file)
    return None

In [10]:
args.dc, args.sbm = False, 'h'

args.nested = True if args.sbm in ['h'] else False

args.force_niter = 40000
args.num_draws = int((1/2) * args.force_niter)

def sbm_name(args):
    dc = f'dc' if args.dc else f'nd'
    dc = f'' if args.sbm in ['m', 'a'] else dc
    file = f'sbm-{dc}-{args.sbm}'
    return file

SBM = sbm_name(args)
SBM

'sbm-nd-h'

In [11]:
def get_membership_matrix(num_rois, df, col='pi'):
    pis = [np.zeros((num_rois, 1)) if np.isnan(pi).all() else pi for pi in df[col]]

    num_modes = len(df)
    num_comms = np.max([pi.shape[-1] for pi in pis])
    num_rois = num_rois
    M = np.zeros((num_rois, num_modes, num_comms)) # membership profile matrix

    for idx_mode, pi in enumerate(pis):
        M[:, idx_mode, :pi.shape[-1]] = pi
    
    return M

In [12]:
marginals_files = sorted(glob.glob(f'{ESTIM_path}/individual/sub-*/partition-modes-group-aligned/{SBM}/desc-mem-mats.pkl', recursive=True))
marginals_df = []
for sbm_file in marginals_files:
    with open(f'{sbm_file}', 'rb') as f:
        row = pickle.load(f)
    marginals_df += [row]
marginals_df = pd.concat(marginals_df).reset_index(drop=True)
mode_ids = list(chain.from_iterable([list(range(count)) for count in marginals_df['sub'].value_counts().sort_index().to_list()]))
marginals_df['mode_id'] = mode_ids
marginals_df

,sub,sbm,pi_0_aligned,pi_1_aligned,pi_2_aligned,pi_3_aligned,pi_4_aligned,pi_5_aligned,pi_6_aligned,pi_7_aligned,pi_8_aligned,omega,mode_id
0,SLC01,sbm-nd-h,"[[0.9887005649717514, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.9359379488652687, 0.005585878898145489, 0....","[[0.930047423159928, 0.0689598675440289, 0.000...","[[0.9834189123072902, 0.016581087692710064], [...","[[0.9986182426922744, 0.0013817573077258387], ...","[[1.0000000000000002], [0.9999999999999999], [...","[[1.0000000000000002], [0.9999999999999999], [...","[[1.0000000000000002], [0.9999999999999999], [...","[[1.0000000000000002], [0.9999999999999999], [...",0.177295,0
1,SLC01,sbm-nd-h,"[[0.9924242424242424, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.9310146923783288, 0.030073461891643713, 0....","[[0.9138235946904278, 0.08582466128530745, 0.0...","[[0.9869962634416201, 0.013003736558379917], [...","[[1.0], [1.0], [1.0], [1.0], [1.0], [0.9999999...","[[1.0], [1.0], [1.0], [1.0], [1.0], [0.9999999...","[[1.0], [1.0], [1.0], [1.0], [1.0], [0.9999999...","[[1.0], [1.0], [1.0], [1.0], [1.0], [0.9999999...","[[1.0], [1.0], [1.0], [1.0], [1.0], [0.9999999...",0.132086,1
2,SLC01,sbm-nd-h,"[[0.9918032786885246, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.9911314162859447, 0.0, 0.0083310937919914,...","[[0.942153748551641, 0.049652834378207875, 0.0...","[[0.9897684668039898, 0.0102315331960104], [0....","[[0.9979536933607981, 0.0020463066392020796], ...","[[1.0000000000000002], [1.0], [1.0], [1.0], [1...","[[1.0000000000000002], [1.0], [1.0], [1.0], [1...","[[1.0000000000000002], [1.0], [1.0], [1.0], [1...","[[1.0000000000000002], [1.0], [1.0], [1.0], [1...",0.122464,2
3,SLC01,sbm-nd-h,"[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.978494623655914, 0.0, 0.021505376344086023...","[[0.9556018036767258, 0.03341426754538097, 0.0...","[[0.9886639548450163, 0.01133604515498376], [0...","[[0.9981106591408362, 0.0018893408591639602], ...","[[1.0000000000000002], [0.9999999999999999], [...","[[1.0000000000000002], [0.9999999999999999], [...","[[1.0000000000000002], [0.9999999999999999], [...","[[1.0000000000000002], [0.9999999999999999], [...",0.093199,3
4,SLC01,sbm-nd-h,"[[0.4782608695652174, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.9153526246909989, 0.011887450923367746, 0....","[[0.8920970026110978, 0.09795351233791275, 0.0...","[[0.9745841708956559, 0.025415829104343764], [...","[[0.9999999999999997], [0.9999999999999999], [...","[[0.9999999999999997], [0.9999999999999999], [...","[[0.9999999999999997], [0.9999999999999999], [...","[[0.9999999999999997], [0.9999999999999999], [...","[[0.9999999999999997], [0.9999999999999999], [...",0.091858,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
69,SLC10,sbm-nd-h,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.03, 0.0, 0.965, 0.004775, 0.0, 0.0, 0.0, 0...","[[0.9599766855670102, 0.034953587628865974, 9....","[[0.9916111389690722, 0.008388861030927834], [...","[[1.0], [1.0], [0.9999999999999999], [1.0], [1...","[[1.0], [1.0], [0.9999999999999999], [1.0], [1...","[[1.0], [1.0], [0.9999999999999999], [1.0], [1...","[[1.0], [1.0], [0.9999999999999999], [1.0], [1...","[[1.0], [1.0], [0.9999999999999999], [1.0], [1...",0.200220,2
70,SLC10,sbm-nd-h,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.007142857142...","[[0.009285714285714286, 0.0, 0.983673469387755...","[[0.9492891635379744, 0.050709816053862454, 1....","[[0.9895083139198907, 0.010491686080109475], [...","[[1.0000000000000002], [1.0000000000000002], [...","[[1.0000000000000002], [1.0000000000000002], [...","[[1.0000000000000002], [1.0000000000000002], [...","[[1.0000000000000002], [1.0000000000000002], [...","[[1.0000000000000002], [1.0000000000000002], [...",0.140248,3
71,SLC10,sbm-nd-h,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.1774193548387097, 0.0, 0.8225806451612904,...","[[0.9793400971210545, 0.020659902878945542], [...","[[0.9979340097121056, 0.002065990287894554], [...","[[1.0], [1.0], [1.0000000000000002], [1.0], [0...","[[1.0], [1.0], [1.0000000000000002], [1.0], [0...",

In [13]:
cols = [col for col in  list(marginals_df.columns) if 'pi_' in col]
cols

['pi_0_aligned',
 'pi_1_aligned',
 'pi_2_aligned',
 'pi_3_aligned',
 'pi_4_aligned',
 'pi_5_aligned',
 'pi_6_aligned',
 'pi_7_aligned',
 'pi_8_aligned']

In [14]:
# SOFT MARGINALS
soft_marginals_df = []
for sub, group in marginals_df.groupby('sub'):
    omegas = group['omega'].to_list()
    dct = {'sub': [sub], 'sbm': [SBM]}
    for col in cols:
        M = get_membership_matrix(args.num_rois, group, col=col)
        SCs = np.average(M, axis=1, weights=omegas) # soft-comms.
        SCs = np.round(SCs, decimals=3)
        dct[col] = [SCs]
    soft_marginals_df += [pd.DataFrame(dct)]
soft_marginals_df = pd.concat(soft_marginals_df).reset_index(drop=True)
soft_marginals_df

,sub,sbm,pi_0_aligned,pi_1_aligned,pi_2_aligned,pi_3_aligned,pi_4_aligned,pi_5_aligned,pi_6_aligned,pi_7_aligned,pi_8_aligned
0,SLC01,sbm-nd-h,"[[0.875, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0....","[[0.943, 0.008, 0.031, 0.0, 0.0, 0.0, 0.0, 0.0...","[[0.928, 0.065, 0.005, 0.0, 0.001, 0.001], [0....","[[0.987, 0.013], [0.979, 0.021], [0.98, 0.02],...","[[0.999, 0.001], [0.997, 0.003], [0.998, 0.002...","[[1.0, 0.0], [0.999, 0.001], [0.999, 0.001], [...","[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1....","[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1....","[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1...."
1,SLC03,sbm-nd-h,"[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.323, 0.0, 0.014, 0.0, 0.0, 0.0, 0.0, 0.663...","[[0.583, 0.361, 0.021, 0.0, 0.006, 0.0, 0.0, 0...","[[0.948, 0.051, 0.0, 0.0, 0.001], [0.966, 0.03...","[[0.99, 0.01], [0.994, 0.006], [0.993, 0.007],...","[[0.999, 0.001], [0.999, 0.001], [0.999, 0.001...","[[0.999, 0.0, 0.001], [1.0, 0.0, 0.0], [1.0, 0...","[[0.999, 0.0, 0.001], [1.0, 0.0, 0.0], [1.0, 0...","[[0.999, 0.001], [1.0, 0.0], [1.0, 0.0], [1.0,..."
2,SLC04,sbm-nd-h,"[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.019, 0.193, 0.08, 0.0, 0.0, 0.0, 0.0, 0.00...","[[0.851, 0.127, 0.018, 0.005, 0.0, 0.0, 0.0, 0...","[[0.98, 0.02, 0.0], [0.979, 0.021, 0.0], [0.98...","[[0.997, 0.003], [0.997, 0.003], [0.998, 0.002...","[[1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0...","[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1....","[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1....","[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1...."
3,SLC05,sbm-nd-h,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.01, 0.96, 0.0, 0.0, 0.0, 0.0, 0.0, 0.027, ...","[[0.892, 0.102, 0.006, 0.0, 0.0, 0.0], [0.893,...","[[0.98, 0.02], [0.98, 0.02], [0.98, 0.02], [0....","[[0.995, 0.005], [0.995, 0.005], [0.995, 0.005...","[[0.999, 0.001], [0.999, 0.001], [0.999, 0.001...","[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1....","[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1....","[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1...."
4,SLC06,sbm-nd-h,"[[0.014, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0....","[[0.992, 0.0, 0.0, 0.0, 0.008, 0.0, 0.0, 0.0, ...","[[0.912, 0.071, 0.015, 0.0, 0.002, 0.0, 0.0, 0...","[[0.985, 0.015], [0.967, 0.033], [0.967, 0.033...","[[0.998, 0.002], [0.993, 0.007], [0.994, 0.006...","[[0.999, 0.001], [0.998, 0.002], [0.998, 0.002...","[[1.0, 0.0, 0.0], [0.999, 0.0, 0.001], [0.999,...","[[1.0, 0.0, 0.0], [0.999, 0.0, 0.001], [0.999,...","[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1...."
5,SLC07,sbm-nd-h,"[[0.002, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0....","[[0.006, 0.989, 0.0, 0.0, 0.0, 0.0, 0.0, 0.005...","[[0.875, 0.122, 0.002, 0.0, 0.0, 0.0, 0.0, 0.0...","[[0.981, 0.019], [0.981, 0.019], [0.981, 0.019...","[[0.995, 0.005], [0.995, 0.005], [0.995, 0.005...","[[0.999, 0.001], [0.999, 0.001], [0.999, 0.001...","[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1....","[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1....","[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1...."
6,SLC08,sbm-nd-h,"[[0.008, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0....","[[0.062, 0.844, 0.0, 0.0, 0.0, 0.0, 0.0, 0.001...","[[0.879, 0.096, 0.023, 0.001, 0.0, 0.0, 0.0, 0...","[[0.981, 0.018, 0.001], [0.981, 0.018, 0.001],...","[[0.996, 0.004], [0.996, 0.004], [0.996, 0.004...","[[0.999, 0.001], [0.999, 0.001], [0.999, 0.001...","[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1....","[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1....","[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1...."
7,SLC09,sbm-nd-h,"[[0.002, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0....","[[0.865, 0.0, 0.06, 0.043, 0.004, 0.025, 0.003...","[[0.934, 0.05, 0.011, 0.001, 0.005, 0.0, 0.0, ...","[[0.992, 0.008, 0.0], [0.971, 0.029, 0.0], [0....","[[0.998, 0.002], [0.995, 0.005], [0.995, 0.005...","[[1.0, 0.0], [0.999, 0.001], [0.999, 0.001], [...","[[1.0, 0.0, 0.0], [0.999, 0.0, 0.001], [0.999,...","[[1.0, 0.0, 0.0], [0.999, 0.0, 0.001], [0.999,...","[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1

In [15]:
for idx, row in marginals_df.iterrows():
    for col in cols[:]:
        pi = row[col]
        if np.any(np.isnan(pi)): continue
        mode_id = row['mode_id']
        sub = row['sub']
        level = col.split('_')[1]
        if level == 'aligned': level = -1

        # data resolution
        folder = (
            f'{ESTIM_path}/individual/sub-{sub}/partition-modes-group-aligned'
            f'/{SBM}/marginal-visuals/res-{args.resolution}/nii'
        )
        print(sub, mode_id)
        marginal_to_nifti(args, parcels_img, pi, mode_id, folder, level=level)

        # # higher resolution
        # folder = (
        #     f'{ESTIM_path}/individual/sub-{sub}/partition-modes-group-aligned'
        #     f'/{SBM}/marginal-visuals/res-{args.high_res}/nii'
        # )
        # marginal_to_nifti(args, parcels_hr_img, pi, mode_id, folder, level=level)

SLC01 0
250828-21:23:18,702 nipype.interface INFO:
	 stderr 2025-08-28T21:23:18.702722:++ 3dTcat: AFNI version=AFNI_20.2.18 (Sep 17 2020) [64-bit]
250828-21:23:18,728 nipype.interface INFO:
	 stderr 2025-08-28T21:23:18.728424:*+ WARNING: can't apply -rlt option -- Not enough points per input dataset.
250828-21:23:18,730 nipype.interface INFO:
	 stderr 2025-08-28T21:23:18.730855:*+ WARNING: Set TR of output dataset to 1.0 s
250828-21:23:18,895 nipype.interface INFO:
	 stderr 2025-08-28T21:23:18.895629:++ elapsed time = 0.2 s
SLC01 0
250828-21:23:20,137 nipype.interface INFO:
	 stderr 2025-08-28T21:23:20.136858:++ 3dTcat: AFNI version=AFNI_20.2.18 (Sep 17 2020) [64-bit]
250828-21:23:20,154 nipype.interface INFO:
	 stderr 2025-08-28T21:23:20.154658:*+ WARNING: can't apply -rlt option -- Not enough points per input dataset.
250828-21:23:20,157 nipype.interface INFO:
	 stderr 2025-08-28T21:23:20.157535:*+ WARNING: Set TR of output dataset to 1.0 s
250828-21:23:20,282 nipype.interface INFO:


In [16]:
for col in cols[:2]:
    M = get_membership_matrix(args.num_rois, soft_marginals_df, col=col)
    grp_pi = np.median(M, axis=1)
    
    level = col.split('_')[1]
    if level == 'aligned': level = -1

    # data resolution
    folder = (
        f'{ESTIM_path}/group/membership-mats-group-aligned'
        f'/{SBM}/marginal-visuals/res-{args.resolution}/nii'
    ) 
    os.system(f'mkdir -p {folder}')
    marginal_to_nifti(args, parcels_img, grp_pi, 0, folder, level)

    # # higher resolution
    # folder = (
    #     f'{ESTIM_path}/group/membership-mats-group-aligned'
    #     f'/{SBM}/marginal-visuals/res-{args.high_res}/nii'
    # ) 
    # os.system(f'mkdir -p {folder}')
    # marginal_to_nifti(args, parcels_hr_img, grp_pi, 0, folder, level)
    # break


250828-21:29:39,611 nipype.interface INFO:
	 stderr 2025-08-28T21:29:39.611475:++ 3dTcat: AFNI version=AFNI_20.2.18 (Sep 17 2020) [64-bit]
250828-21:29:39,614 nipype.interface INFO:
	 stderr 2025-08-28T21:29:39.614347:++ WARNING: file /home/govindas/.afni.log is now 101309059 (101 million) bytes long!
250828-21:29:39,637 nipype.interface INFO:
	 stderr 2025-08-28T21:29:39.637762:*+ WARNING: can't apply -rlt option -- Not enough points per input dataset.
250828-21:29:39,641 nipype.interface INFO:
	 stderr 2025-08-28T21:29:39.641202:*+ WARNING: Set TR of output dataset to 1.0 s
250828-21:29:39,816 nipype.interface INFO:
	 stderr 2025-08-28T21:29:39.816100:++ elapsed time = 0.2 s
250828-21:29:41,50 nipype.interface INFO:
	 stderr 2025-08-28T21:29:41.050639:++ 3dTcat: AFNI version=AFNI_20.2.18 (Sep 17 2020) [64-bit]
250828-21:29:41,53 nipype.interface INFO:
	 stderr 2025-08-28T21:29:41.053285:++ WARNING: file /home/govindas/.afni.log is now 101321457 (101 million) bytes long!
250828-21:29:

In [17]:
col.split('_')

['pi', '1', 'aligned']